## Initialization

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 3000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [ ]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'wikiproject-414111'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][316.7 MiB/316.7 MiB]   30.8 MiB/s                                   
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# parquetFile.show()

In [ ]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("id", "text").rdd

# Train word2vec model

In [ ]:
df = doc_text_pairs.toDF()

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)
# Initialize the Porter Stemmer
porter_stemmer = PorterStemmer()

def word_split(text):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stop_tokens = set(tokens).intersection(all_stopwords)
  return [t for t in tokens if t not in stop_tokens]

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# Register the function as a UDF (User Defined Function)
my_udf = udf(word_split, ArrayType(StringType()))

# Apply the function to the 'text' column and create a new column with the list
df_with_list_column = df.withColumn("text", my_udf(df["text"]))

In [ ]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import Word2VecModel

word2Vec = Word2Vec(vectorSize=20, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(df_with_list_column)

In [ ]:
result = model.transform(df_with_list_column)

# Run model on query

In [ ]:
queryDF = spark.createDataFrame([
    ("political").split(),
], ["text"])

# Apply the function to the 'text' column and create a new column with the list
queryDF = queryDF.withColumn("text", my_udf(queryDF["text"]))

In [ ]:
query = model.transform(documentDF)
query_vector = query.select('result').collect()[0]['result']

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, array
from pyspark.sql.types import FloatType

cosine_udf = F.udf(lambda v: float(np.round(np.abs(v.dot(query_vector) / (v.norm(2) * query_vector.norm(2))), 3)),  FloatType())

# Define the vector you want to compare against


# Calculate dot product between each vector and the target vector
result_df = result.withColumn("cosine", cosine_udf('result'))


In [ ]:
ids = result_df.orderBy(desc("cosine")).select('id').limit(100)

In [ ]:
ids.collect()

[Row(id=1193),
 Row(id=1950),
 Row(id=1495),
 Row(id=705),
 Row(id=1111),
 Row(id=1070),
 Row(id=1093),
 Row(id=2151),
 Row(id=307),
 Row(id=1973),
 Row(id=12),
 Row(id=1624),
 Row(id=1935),
 Row(id=691),
 Row(id=1078),
 Row(id=1023),
 Row(id=2371),
 Row(id=881),
 Row(id=1030),
 Row(id=1814),
 Row(id=1943),
 Row(id=863),
 Row(id=2154),
 Row(id=1623),
 Row(id=2187),
 Row(id=2176),
 Row(id=1653),
 Row(id=1921),
 Row(id=2104),
 Row(id=2167),
 Row(id=2314),
 Row(id=2070),
 Row(id=2376),
 Row(id=1235),
 Row(id=600),
 Row(id=1291),
 Row(id=1338),
 Row(id=2235),
 Row(id=771),
 Row(id=1546),
 Row(id=643),
 Row(id=2360),
 Row(id=983),
 Row(id=1857),
 Row(id=569),
 Row(id=339),
 Row(id=1625),
 Row(id=988),
 Row(id=1466),
 Row(id=795),
 Row(id=358),
 Row(id=701),
 Row(id=2025),
 Row(id=1051),
 Row(id=1563),
 Row(id=2448),
 Row(id=746),
 Row(id=640),
 Row(id=1270),
 Row(id=2384),
 Row(id=951),
 Row(id=665),
 Row(id=794),
 Row(id=700),
 Row(id=1938),
 Row(id=1807),
 Row(id=1930),
 Row(id=737),
 Row

# Save model to file

In [ ]:
model.save('model')

In [ ]:
 mod = Word2VecModel.load('model')